# 목차
- [Guidance 소개](../../../../code/01.Introduce)
- [설치](../../../../code/01.Introduce)
- [제약 없는 생성](../../../../code/01.Introduce)
- [Phi 3를 위한 발언](../../../../code/01.Introduce)
- [정규 표현식](../../../../code/01.Introduce)
- [선택](../../../../code/01.Introduce)
- [사고의 흐름](../../../../code/01.Introduce)
- [JSON 생성](../../../../code/01.Introduce)
- [HTML 생성](../../../../code/01.Introduce)


# Guidance 소개
Guidance는 모든 언어 모델(LM)의 출력을 제어할 수 있는 검증된 오픈소스 Python 라이브러리입니다. 하나의 API 호출로, 모델이 따라야 할 정확한 프로그래밍 제약 조건을 Python으로 표현하고, JSON, Python, HTML, SQL 또는 사용 사례에 필요한 구조화된 출력을 생성할 수 있습니다.

Guidance는 기존의 프롬프트 기술과는 다릅니다. 추론 레이어에서 모델을 토큰 단위로 조정하여 제약 조건을 강제하며, 더 높은 품질의 출력을 생성하고, 고도로 구조화된 시나리오에서 비용과 지연 시간을 최대 30–50%까지 줄일 수 있습니다.

Guidance에 대해 더 알아보려면 [GitHub의 공개 저장소](https://github.com/guidance-ai/guidance)를 방문하거나 Microsoft Build에서 열린 [Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs)을 시청하세요.


# 설정
1. `pip install guidance --pre` 명령어로 Guidance를 설치합니다.
2. https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml 페이지로 이동하여 "배포" 버튼을 클릭하여 Azure에 Phi 3.5 mini 엔드포인트를 배포합니다.
3. 엔드포인트의 API 키를 `AZURE_PHI3_KEY`라는 환경 변수에 저장하고, URL을 `AZURE_PHI3_URL`라는 환경 변수에 저장합니다.


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# 제한 없는 생성
`gen()` 함수를 사용하면 아무런 제약 없이 텍스트를 생성할 수 있습니다. 이는 Guidance 없이 모델을 사용하는 것과 동일합니다.

## 채팅 형식
많은 채팅 모델과 마찬가지로, Phi-3는 사용자와 어시스턴트 간의 메시지를 특정 형식으로 기대합니다. Guidance는 Phi-3의 채팅 템플릿을 지원하며 채팅 형식을 관리해줍니다. 채팅 턴을 생성하려면 대화의 각 부분을 `with user()` 또는 `with assistant()` 블록에 넣으세요. 시스템 메시지를 설정하려면 `with system()` 블록을 사용할 수 있습니다.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## 토큰 절약
고도로 구조화된 시나리오에서는 Guidance가 토큰을 건너뛰고 필요한 토큰만 생성하여 성능을 향상시키고 효율성을 높이며 API 비용을 절감할 수 있습니다. 생성된 토큰은 이 노트북에서 강조된 배경으로 표시됩니다. 강제된 토큰은 강조 표시 없이 나타나며 입력 토큰과 동일한 비용이 들며, 이는 출력 토큰 비용의 약 3분의 1로 추정됩니다.

*참고:* 제약 없이 생성된 첫 번째 예시는 우리가 아무런 제약을 제공하지 않았기 때문에 어떤 토큰도 강제할 수 없었습니다.


# Phi 3에 대해 말하기  
Guidance를 사용하면 모델의 응답에 텍스트를 쉽게 삽입할 수 있습니다. 이는 모델의 출력 방향을 특정 방향으로 유도하고 싶을 때 유용합니다.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# 정규식으로 제한하기
이전 예제에서 Phi 3는 `Canberra`로 질문에 답한 후 추가 설명을 제공했습니다. 모델의 출력을 정확히 한 단어로 제한하기 위해 정규식을 사용할 수 있습니다.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

# 여러 선택지 중에서 선택하기
몇 가지 가능한 선택지가 알려져 있을 때, `select()` 함수를 사용하여 모델이 옵션 목록에서 선택하도록 할 수 있습니다.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

`select()`을 사용하면 `Can` 토큰만 생성되었습니다. `Canberra`가 응답을 완성할 수 있는 유일한 옵션이기 때문에 나머지 토큰은 강제로 생성되었습니다.


# 사고의 흐름
사고의 흐름은 문제를 단계별로 처리하도록 유도하여 모델 출력의 품질을 향상시키는 데 도움을 줄 수 있는 기술입니다. 일반적으로 최종 답변에 도달하기 위해서는 여러 번의 프롬프트가 필요합니다. 먼저 모델에게 단계별로 사고하도록 지시합니다. 그런 다음 모델에게 최종 답변을 제공하도록 다시 프롬프트를 전달합니다. 표준 채팅 추론 API를 사용할 경우, 이 과정은 2번의 API 호출이 필요하며, 모델이 생성한 "사고의 흐름"은 두 번 비용이 청구됩니다 – 한 번은 모델이 이를 생성할 때 출력 토큰으로, 또 한 번은 두 번째 호출에서 입력 토큰으로 사용될 때. Guidance를 사용하면 전체 다단계 프로세스가 단일 API 호출로 처리되고 비용과 지연 시간이 줄어듭니다.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON 생성
Guidance는 JSON 스키마 또는 pydantic 모델을 준수하는 JSON 생성을 보장하는 데 사용할 수 있습니다. 여기에서 보여지는 사용자 프로필 스키마와 같은 경우입니다.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML 생성

Guidance는 코드 생성 및 프로그래밍 언어의 문법 요구 사항을 따르는 데 사용할 수 있습니다. 이 섹션에서는 매우 간단한 HTML 웹페이지를 작성하기 위한 작은 Guidance 프로그램을 만들어 보겠습니다.

웹페이지를 더 작은 섹션으로 나누고, 각 섹션은 자체 Guidance 함수를 갖게 됩니다. 그런 다음 최종 함수를 결합하여 HTML 웹페이지를 생성합니다. 이후 이 함수를 Azure AI에서 Guidance를 지원하는 모델에 실행해 보겠습니다.

*참고:* 이것은 완전한 기능을 갖춘 HTML 생성기가 아닙니다. 목표는 개별적인 필요에 맞는 구조화된 출력을 생성하는 방법을 보여주는 것입니다.

우선 Guidance에서 필요한 것을 가져오는 것으로 시작합니다:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML 웹페이지는 매우 구조화되어 있으며, 우리는 Guidance를 사용하여 페이지의 해당 부분을 '강제'할 것입니다.  
모델에서 텍스트를 명시적으로 요구할 때, 태그로 간주될 수 있는 내용을 포함하지 않도록 해야 합니다. 즉, '<'와 '>' 문자를 제외해야 합니다.


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

그런 다음 이 함수를 사용하여 임의의 HTML 태그 내에서 텍스트를 생성할 수 있습니다:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

페이지 헤더를 만들어봅시다.  
이를 위해 페이지 제목을 생성해야 합니다:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTML 페이지의 본문은 제목과 단락으로 채워질 예정입니다.  
각각을 처리하는 함수를 정의할 수 있습니다:  


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

이제 HTML 본문을 정의하는 함수입니다.  
이 함수는 `select()`를 `recurse=True`로 사용하여 여러 제목과 단락을 생성합니다:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

다음으로, 전체 HTML 페이지를 생성하는 함수로 넘어갑니다.  
HTML 시작 태그를 추가한 후, 헤더를 생성하고, 본문을 생성한 다음, 마지막으로 HTML 종료 태그를 추가합니다:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

우리는 사용자 친화적인 래퍼를 제공하며, 이를 통해 다음을 수행할 수 있습니다:
- 생성의 온도를 설정하기
- Model 객체에서 생성된 페이지를 캡처하기


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

그런 다음 출력을 파일에 작성할 수 있습니다:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

그리고 [결과를 확인하세요](../../../../code/01.Introduce/sample_page.html).



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
